In [1]:
# header
import pandas as pd
import ReadLift
import MatchHeadwords
import AddIPAFlex
from datetime import datetime
from FindBib import read_date

In [2]:
# update file pointers in ReadLift for new export folder
in_file = 'Flexport_new/Flexport_new.lift'
out_file = 'Flexport_new/Flexicon_NewData.csv'
senses_file = 'Flexport_new/Senses_NewData.csv'
ReadLift.set_filenames(in_file, out_file, senses_file)

In [3]:
# call main (note tends to take ~20sec to execute)
ReadLift.main()

<function get_entries_df at 0x7f6973809200> 7.043222904205322
<function get_these_vars at 0x7f6973809440> 1.8577079772949219
<function get_senses_df at 0x7f6973809320> 4.473207235336304
<function get_these_vars at 0x7f6973809440> 1.5484461784362793
<function main at 0x7f6973808680> 14.954824686050415


In [4]:
# read Flexicon_NewData.csv
# drop all data that's earlier than recent field trip
flex_df = pd.read_csv(out_file, keep_default_na=False, index_col='entry_id')
cutoff = datetime(2019, 11, 3)
new_data = [read_date(t) > cutoff for t in flex_df['date_modified']]
new_data = flex_df[new_data]
del flex_df
new_data.shape

(382, 10)

In [5]:
# read cleaned data
flexicon = pd.read_csv('flexiconCLEAN.csv', keep_default_na=False, index_col='entry_id')
flexicon.shape

(1501, 10)

In [6]:
# find rows in common
in_flexicon = [i in flexicon.index for i in new_data.index]
new_data.loc[:, 'in_flexicon'] = in_flexicon
in_flexicon.count(True)

32

In [7]:
# read old data
old_flexicon = pd.read_csv('flexicon.csv', keep_default_na=False, index_col='entry_id')
old_flexicon.shape

(1759, 10)

In [8]:
# rows edited by Karol and found in old_flexicon
in_old = [i in old_flexicon.index for i in new_data.index]
new_data.loc[:, 'in_old'] = in_old
in_old.count(True)

65

In [9]:
# entries that Karol has edited but I dropped while cleaning
edited_but_dropped = [old and not new for old, new in zip(in_old, in_flexicon)]
new_data.loc[:, 'edited_but_dropped'] = edited_but_dropped
edited_but_dropped.count(True)

33

In [10]:
edited_but_dropped = new_data[new_data['edited_but_dropped']]
edited_but_dropped['headword'].values

array(['edʹuuk', 'erét', 'tsananaa', 'waa', 'naga hẽ', 'asok', 'hõm',
       'nuu me', 'ji', 'tä', 'hʹỹỹb', 'mäh', 'hẽnh', 'ra', 'wapad',
       'mahang', 'takʹëp', 'aha', 'naëënh', 'né hẽ', 'soo', 'hadoo', 'sa',
       'adyyk', 'is', 'kyyh', 'biin', 'manäh', 'kapỹỹj', 'ỹỹm', 'ahỹỹh',
       'asoop', 'jajé'], dtype=object)

In [13]:
# find any words in edited_but_dropped that might be copies of entries in flexiconCLEAN

flexicon['ipa'] = [AddIPAFlex.to_ipa(x, bib='eppsob') for x in flexicon['headword']]
for index, row in flexicon.copy().iterrows():
    if 'Predicted phonemic form from source' in row['note']:
        flexicon.at[index, 'ipa'] = row['headword']

edited_but_dropped['ipa'] = [AddIPAFlex.to_ipa(x, bib='sil') for x in edited_but_dropped['headword']]
        
matches = MatchHeadwords.match_dfs(edited_but_dropped, flexicon)
matches

/home/mark/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


KeyError: 'entry_id'